# Case Técnico – iFood  
### Notebook de Processamento de Dados, Teste A/B, Segmentação RFM e Viabilidade Financeira

**Autor:** Daniel Leite  
**Ano:** 2025  
**Repositório GitHub:** [github.com/danielmelo2025/Case-iFood](https://github.com/danielmelo2025/Case-iFood)

---

## O que você encontrará neste notebook

Este notebook implementa toda a solução técnica solicitada no Case de Data Analysis do iFood:

1. **ETL completo**  
   - Download dos arquivos (.gz e .tar.gz)  
   - Descompressão  
   - Leitura otimizada dos 3.6M pedidos  
   - Preparação das tabelas finais

2. **Preparação das bases analíticas**  
   - Orders  
   - Consumers  
   - Restaurants  
   - AB Test Reference  

3. **Análise Estatística do Teste A/B**  
   - Receita por usuário  
   - Ticket médio  
   - Frequência de pedidos  
   - Retenção  
   - Testes estatísticos (T-Test, Z-Test)

4. **Segmentação RFM (Recency, Frequency, Monetary)**  
   - Atribuição de scores  
   - Criação dos segmentos  
   - Análise do impacto da campanha por segmento

5. **Viabilidade Financeira (Simulação Monte Carlo)**  
   - Premissas  
   - Distribuições  
   - ROI  
   - Cenários pessimista / base / otimista  
---

## Observação Importante

Este notebook foi desenvolvido no **Google Colab**, ambiente recomendado pelo case por garantir:  
- Reprodutibilidade  
- Disponibilidade de recursos  
- Facilidade de leitura de grandes volumes de dados  

Todas as etapas podem ser executadas sem configuração adicional.

---

In [1]:
# ========================================
# Imports
# ========================================
import pandas as pd
import numpy as np
import json
import gzip
import tarfile
import requests
from io import BytesIO

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import ttest_ind, chi2_contingency
from statsmodels.stats.proportion import proportions_ztest
from scipy import stats

pd.set_option("display.max_columns", None)

# 1. Download dos arquivos  
Os dados originais estão compactados (`.gz` e `.tar.gz`).  
Vamos baixá-los localmente.


In [2]:
# ========================================
# Função de Download
# ========================================
def download_to_disk(url, filename):
    print(f"Baixando {filename} ...")
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(filename, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024*1024):
                f.write(chunk)
    print(f"Arquivo salvo em: {filename}")

In [3]:
# ========================================
# URLs oficiais do case
# ========================================
url_orders = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"
url_consumers = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
url_restaurants = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
url_ab = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"

# Downloads
download_to_disk(url_orders, "order.json.gz")
download_to_disk(url_consumers, "consumer.csv.gz")
download_to_disk(url_restaurants, "restaurant.csv.gz")
download_to_disk(url_ab, "ab_test_ref.tar.gz")

Baixando order.json.gz ...
Arquivo salvo em: order.json.gz
Baixando consumer.csv.gz ...
Arquivo salvo em: consumer.csv.gz
Baixando restaurant.csv.gz ...
Arquivo salvo em: restaurant.csv.gz
Baixando ab_test_ref.tar.gz ...
Arquivo salvo em: ab_test_ref.tar.gz


# 2. Leitura e ETL das bases  
## 2.1 Leitura otimizada do `order.json.gz`  
Usamos leitura em *chunks* para não estourar memória.

In [ ]:
# ========================================
# Leitura otimizada do orders.json.gz
# ========================================
def read_orders_in_chunks(filepath, chunksize=200_000, usecols=None):
    with gzip.open(filepath, 'rt') as f:
        for chunk in pd.read_json(f, lines=True, chunksize=chunksize):
            if usecols:
                chunk = chunk[usecols]
            yield chunk

orders_usecols = [
    'cpf', 'customer_id', 'delivery_address_city', 'delivery_address_country',
    'delivery_address_district','delivery_address_state', 'merchant_id',
    'merchant_timezone', 'order_created_at', 'order_id',
    'order_scheduled', 'order_total_amount', 'origin_platform',
    'order_scheduled_date'
]

orders_list = []

for chunk in read_orders_in_chunks(
        "/content/order.json.gz",
        chunksize=150_000,
        usecols=orders_usecols):

    chunk["order_created_at"] = pd.to_datetime(chunk["order_created_at"])
    orders_list.append(chunk)

orders = pd.concat(orders_list, ignore_index=True)

orders.info()
orders.head()

## 2.2 Leitura das demais bases (mais leves)

In [ ]:
consumers = pd.read_csv("/content/consumer.csv.gz", compression="gzip")
restaurants = pd.read_csv("/content/restaurant.csv.gz", compression="gzip")

## 2.3 Leitura da base A/B (`ab_test_ref.tar.gz`)  

In [ ]:
def load_ab_ref_from_tar(tar_path):
    with tarfile.open(tar_path, "r:gz") as tar:

        members = [
            m for m in tar.getmembers()
            if m.name.endswith(".csv") and not m.name.startswith("._")
        ]

        if not members:
            raise ValueError("Nenhum CSV válido encontrado dentro do TAR.")

        member = members[0]
        print(f"Lendo arquivo real: {member.name}")

        f = tar.extractfile(member)
        raw = f.read()

        for enc in ["utf-8", "latin1", "cp1252"]:
            try:
                df = pd.read_csv(BytesIO(raw), encoding=enc)
                print(f"Lido com encoding: {enc}")
                return df
            except:
                pass

        raise ValueError("Não foi possível ler com encodings testados.")

ab_ref = load_ab_ref_from_tar("/content/ab_test_ref.tar.gz")
ab_ref.head()

# 3. Preparação das Bases



In [ ]:
def prepare_restaurants(restaurants):
    df = restaurants.rename(columns={'id': 'restaurant_id'})
    df = df.drop_duplicates(subset=['restaurant_id'])
    df = df[['restaurant_id','merchant_city','merchant_state','enabled','price_range',
             'average_ticket','delivery_time','minimum_order_value','merchant_zip_code',
             'merchant_country','created_at']]
    df['enabled'] = df['enabled'].astype(bool)
    df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
    return df

def prepare_consumers(consumers):
    df = consumers.rename(columns={'customerid': 'customer_id'})
    df = df.drop_duplicates(subset=['customer_id'])
    df['active'] = df.get('active', pd.Series([True]*df.shape[0])).astype(bool)
    if 'createdat' in df.columns:
        df['createdat'] = pd.to_datetime(df['createdat'], errors='coerce')
    return df

def prepare_orders(orders):
    df = orders.copy()
    df['order_created_at'] = pd.to_datetime(df['order_created_at'], errors='coerce')
    df = df.rename(columns={'merchant_id': 'restaurant_id'})
    df = df[['order_id','customer_id','restaurant_id','order_created_at','order_total_amount','order_scheduled']]
    df = df.dropna(subset=['order_id','customer_id','restaurant_id','order_created_at'])
    return df

def prepare_ab_ref(ab_ref):
    df = ab_ref.rename(columns={'is_target': 'ab_group'})
    df['ab_group'] = df['ab_group'].astype('category')
    df = df.drop_duplicates(subset=['customer_id'])
    return df

In [ ]:
restaurants = prepare_restaurants(restaurants)
consumers = prepare_consumers(consumers)
orders = prepare_orders(orders)
ab_ref = prepare_ab_ref(ab_ref)

In [ ]:
# Base final unificada
orders_consumers = orders.merge(consumers, on='customer_id', how='left')
orders_consumers_restaurants = orders_consumers.merge(restaurants, on='restaurant_id', how='left')
final_df = orders_consumers_restaurants.merge(ab_ref, on='customer_id', how='left')

final_df.head()

# 4. Análise do Teste A/B
- Receita total por usuário  
- Número de pedidos  
- Ticket médio  
- Significância estatística  
- Intervalos de confiança

## 4.1 Receita total por usuário  

In [ ]:
# Criar dataset de métricas por usuário
user_metrics = final_df.groupby(['ab_group', 'customer_id']).agg(
    total_revenue=('order_total_amount', 'sum'),
    total_orders=('order_id', 'nunique')
).reset_index()

# Ticket médio
user_metrics['avg_ticket'] = np.where(
    user_metrics['total_orders'] > 0,
    user_metrics['total_revenue'] / user_metrics['total_orders'],
    np.nan
)

# Medidas resumo por grupo
summary = user_metrics.groupby('ab_group')[['total_revenue', 'total_orders', 'avg_ticket']].describe()
display(summary)

In [ ]:
# Receitas por usuário dos grupos
control_revenue = user_metrics[user_metrics['ab_group'] == 'control']['total_revenue']
target_revenue = user_metrics[user_metrics['ab_group'] == 'target']['total_revenue']

# Teste t para média independente (não assume variâncias iguais)
t_stat, p_val = stats.ttest_ind(target_revenue, control_revenue, equal_var=False)

# Médias e desvios padrões
mean_c = control_revenue.mean()
mean_t = target_revenue.mean()
std_c = control_revenue.std()
std_t = target_revenue.std()
n_c = control_revenue.count()
n_t = target_revenue.count()

# Diferença média
diff_mean = mean_t - mean_c

# Erro padrão da diferença
se_diff = np.sqrt(std_c**2/n_c + std_t**2/n_t)

# Graus de liberdade pelo método de Welch
df = (std_c**2/n_c + std_t**2/n_t)**2 / ((std_c**2/n_c)**2 / (n_c-1) + (std_t**2/n_t)**2 / (n_t-1))

# Intervalo de confiança (95%)
alpha = 0.05
t_crit = stats.t.ppf(1-alpha/2, df)
ci_lower = diff_mean - t_crit * se_diff
ci_upper = diff_mean + t_crit * se_diff

print(f"Teste t: t_stat = {t_stat:.4f}, p-value = {p_val:.4e}")
print(f"Diferença média entre grupos: {diff_mean:.2f}")
print(f"Intervalo de confiança 95% da diferença: [{ci_lower:.2f}, {ci_upper:.2f}]")

## 4.2 Ticket Médio

In [ ]:
# Verificar equilíbrio dos grupos
print(final_df['ab_group'].value_counts())

# Função para calcular KPIs
def calculate_metrics(df):
    grouped = df.groupby('ab_group')
    n_users = grouped['customer_id'].nunique()
    n_orders = grouped['order_id'].nunique()
    avg_order_value = grouped['order_total_amount'].mean()
    total_revenue = grouped['order_total_amount'].sum()
    orders_per_user = n_orders / n_users

    metrics = pd.DataFrame({
        'ab_group': n_users.index,
        'n_users': n_users.values,
        'n_orders': n_orders.values,
        'avg_order_value': avg_order_value.values,
        'orders_per_user': orders_per_user.values,
        'total_revenue': total_revenue.values
    })
    return metrics


metrics_df = calculate_metrics(final_df)
print(metrics_df)

# Teste estatístico para ticket médio
control = final_df.loc[final_df['ab_group'] == 'control', 'order_total_amount'].dropna()
target = final_df.loc[final_df['ab_group'] == 'target', 'order_total_amount'].dropna()

t_stat, p_value = stats.ttest_ind(target, control, equal_var=False)
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Calcular lift percentual
avg_control = control.mean()
avg_target = target.mean()
lift = (avg_target - avg_control) / avg_control * 100
print(f"Lift percentual no ticket médio (target vs control): {lift:.2f}%")

# Interpretação simples
if p_value < 0.05:
    print("Diferença estatisticamente significativa no ticket médio entre grupos.")
else:
    print("Não há diferença estatisticamente significativa no ticket médio.")

In [ ]:
def mean_ci(data, conf=0.95):
    n = len(data)
    mean = np.mean(data)
    std = np.std(data, ddof=1)
    h = std / np.sqrt(n) * stats.t.ppf((1 + conf) / 2, n - 1)
    return mean, mean - h, mean + h

# Calcular para cada grupo
avg_control, ci_control_low, ci_control_high = mean_ci(control)
avg_target, ci_target_low, ci_target_high = mean_ci(target)

print(f"Ticket médio controle: {avg_control:.2f}, IC95%: [{ci_control_low:.2f}, {ci_control_high:.2f}]")
print(f"Ticket médio target: {avg_target:.2f}, IC95%: [{ci_target_low:.2f}, {ci_target_high:.2f}]")

## 4.3 Retenção

In [ ]:
final_df['order_created_at'] = pd.to_datetime(final_df['order_created_at'])

final_df['order_ym'] = final_df['order_created_at'].dt.to_period('M')

# Primeiro mês de pedido por usuário
first_order = final_df.groupby('customer_id')['order_ym'].min().reset_index()
first_order = first_order.rename(columns={'order_ym': 'first_order_ym'})

# Juntar info de primeiro pedido
df = final_df.merge(first_order, on='customer_id')

# Criar coluna com meses desde o primeiro pedido
df['months_since_first'] = (df['order_ym'] - df['first_order_ym']).apply(lambda x: x.n)

# Para cada grupo A/B calcular % de usuários ativos em cada mês
retention = df.groupby(['ab_group', 'months_since_first'])['customer_id'].nunique().reset_index()

# Total de usuários que fizeram primeiro pedido no grupo, para base de cálculo (%)
total_users = first_order.merge(ab_ref, on='customer_id').groupby('ab_group')['customer_id'].nunique().reset_index(name='total_users')

# Juntar totais para calcular taxa de retenção
retention = retention.merge(total_users, on='ab_group')
retention['retention_rate'] = retention['customer_id'] / retention['total_users']

print(retention.head(20))

In [ ]:
# Dados para o gráfico
retention_month_1 = retention[retention['months_since_first'] == 1][['ab_group', 'retention_rate']]

fig, ax = plt.subplots(figsize=(6, 5))

colors = ['#c6f5c6', '#2ca02c']  # Primeiro controle, depois target
bars = ax.bar(retention_month_1['ab_group'], retention_month_1['retention_rate'], color=colors)

ax.set_ylim(0, 1)
ax.set_ylabel('Taxa de Retenção no Mês 1')
ax.set_title('Comparação de Retenção Mensal por Grupo')

# Anotações das taxas nas barras
ax.bar_label(bars, fmt='%.2f')

# Anotar intervalo de confiança e p-valor no gráfico na posição desejada
textstr = '\n'.join((
    'Diferença 95% CI: [0.0263, 0.0378]',
    'z-statistic: -30.9',
    'p-value < 0.00001 (significativo)'
))
ax.text(0.3, 0.85, textstr, transform=ax.transAxes, fontsize=10,
        bbox=dict(boxstyle='round,pad=0.5', facecolor='#f3f3f3', alpha=0.9))

plt.show()

In [ ]:
retention_month_1

# 5 Viabilidade financeira da campanha (Simulação Monte Carlo)

Premissas:
- Número de usuários target ≈ 806.466 (do experimento)
- Custo unitário médio do cupom: média R$10, desvio R$3
- Receita incremental média por usuário: média R$35, desvio R$10
- Custo operacional: média 15% do custo, desvio 5%

In [ ]:
# Número de usuários alvo
n_users_target = 806466

# Semente para reprodutibilidade
np.random.seed(42)

# Distribuições para os parâmetros (normal truncada para evitar valores negativos)
custo_unit = np.clip(np.random.normal(10, 3, 10000), 1, None)  # custo médio 10, desvio 3
receita_inc = np.clip(np.random.normal(35, 10, 10000), 5, None)  # receita incremental média 35, desvio 10
custo_op = np.clip(np.random.normal(0.15, 0.05, 10000), 0, 1)  # custo operacional média 15%, desvio 5%

# Simulação de Monte Carlo do ROI
roi_sim = []
for i in range(10000):
    custo_total = n_users_target * custo_unit[i]
    custo_operacional = custo_total * custo_op[i]
    receita_incremental = n_users_target * receita_inc[i]
    investimento_total = custo_total + custo_operacional
    roi = (receita_incremental - investimento_total) / investimento_total
    roi_sim.append(roi)

roi_sim = np.array(roi_sim)

# Estatísticas
roi_mean = roi_sim.mean()
roi_std = roi_sim.std()
roi_ci_low = np.percentile(roi_sim, 2.5)
roi_ci_high = np.percentile(roi_sim, 97.5)

# Visualização do resultado
plt.hist(roi_sim, bins=50, edgecolor='k', alpha=0.6)
plt.axvline(roi_mean, color='r', linestyle='dashed', linewidth=2, label=f'Média ROI: {roi_mean:.2f}')
plt.axvline(roi_ci_low, color='g', linestyle='dashed', linewidth=1, label=f'IC 2.5%: {roi_ci_low:.2f}')
plt.axvline(roi_ci_high, color='g', linestyle='dashed', linewidth=1, label=f'IC 97.5%: {roi_ci_high:.2f}')
plt.title('Simulação Monte Carlo do ROI da Campanha de Cupom')
plt.xlabel('ROI')
plt.ylabel('Frequência')
plt.legend()
plt.show()

print(f'Média do ROI: {roi_mean:.2f}')
print(f'Desvio padrão do ROI: {roi_std:.2f}')
print(f'Intervalo de confiança 95% do ROI: [{roi_ci_low:.2f}, {roi_ci_high:.2f}]')

In [ ]:
n_users_target = 806466

cenarios = {
    'Pessimista': {'custo_unit': 15, 'receita_inc': 20, 'custo_op': 0.20},
    'Base': {'custo_unit': 10, 'receita_inc': 35, 'custo_op': 0.15},
    'Otimista': {'custo_unit': 7, 'receita_inc': 50, 'custo_op': 0.10}
}

results = []

for nome, params in cenarios.items():
    custo_total = n_users_target * params['custo_unit']
    custo_operacional = custo_total * params['custo_op']
    receita_incremental = n_users_target * params['receita_inc']
    investimento_total = custo_total + custo_operacional
    roi = (receita_incremental - investimento_total) / investimento_total
    results.append({
        'Cenário': nome,
        'Receita Incremental Total (R$)': receita_incremental,
        'Custo Total (cupom + operacional) (R$)': investimento_total,
        'ROI': roi
    })

results_df = pd.DataFrame(results)
results_df['ROI %'] = results_df['ROI'] * 100
results_df['Receita Incremental Total (R$)'] = results_df['Receita Incremental Total (R$)'].map('{:,.2f}'.format)
results_df['Custo Total (cupom + operacional) (R$)'] = results_df['Custo Total (cupom + operacional) (R$)'].map('{:,.2f}'.format)
results_df['ROI %'] = results_df['ROI %'].map('{:.2f}%'.format)
results_df.drop(columns=['ROI'], inplace=True)

display(results_df)

# 6 Segmentação RFM

In [ ]:
# Garantir formato de data
orders["order_created_at"] = pd.to_datetime(orders["order_created_at"])

# Consolidar métricas por cliente
rfm_base = (
    orders.groupby("customer_id")
    .agg(
        last_order_date = ("order_created_at", "max"),
        order_count     = ("order_id", "count"),
        total_spent     = ("order_total_amount", "sum")
    )
    .reset_index()
)

rfm_base.head()

In [ ]:
def calculate_rfm(df, ref_date, n_bins=4):
    df = df.copy()

    # ----------------------------
    # Calcular recência
    # ----------------------------
    df['last_order_date'] = pd.to_datetime(df['last_order_date'])
    df['recency'] = (ref_date - df['last_order_date']).dt.days

    def safe_qcut(series, q):
        # 1) Tenta qcut normal
        try:
            return pd.qcut(series, q, labels=False, duplicates='drop') + 1
        except:
            pass

        # 2) Tenta qcut com rank
        try:
            ranked = series.rank(method='first')
            return pd.qcut(ranked, q, labels=False, duplicates='drop') + 1
        except:
            pass

        # 3) Tenta cut
        try:
            return pd.cut(series, q, labels=False) + 1
        except:
            pass

        # 4) Fallback: tudo score 1
        return pd.Series([1] * len(series), index=series.index)

    # ----------------------------
    # Calcular scores
    # ----------------------------
    df['recency_score']   = safe_qcut(df['recency'], n_bins)
    df['frequency_score'] = safe_qcut(df['order_count'], n_bins)
    df['monetary_score']  = safe_qcut(df['total_spent'], n_bins)

    # Recência invertida (poucos dias = score maior)
    df['recency_score'] = (n_bins + 1) - df['recency_score']

    # ----------------------------
    # Criar segmentos
    # ----------------------------
    def assign_segment(row):
        r, f, m = row['recency_score'], row['frequency_score'], row['monetary_score']

        if r >= n_bins and f >= n_bins and m >= n_bins-1:
            return "heavy_user"
        if r >= n_bins-1 and f <= 2:
            return "leve_engajado"
        if m == n_bins:
            return "high_spender"
        if 2 <= r < n_bins-1:
            return "adormecido"
        if r == 1:
            return "churnado"

        return "outros"

    df['segmento'] = df.apply(assign_segment, axis=1)

    return df

In [ ]:
ref_date = orders["order_created_at"].max()

rfm_df = calculate_rfm(rfm_base, ref_date)

In [ ]:
rfm_df["segmento"].value_counts()

In [ ]:
rfm_df.head()

In [ ]:
segment_profile = (
    rfm_df.groupby("segmento")
    .agg(
        usuarios = ("customer_id", "nunique"),
        recencia_media = ("recency", "mean"),
        freq_media = ("order_count", "mean"),
        ticket_medio = ("total_spent", "mean"),
        gasto_total = ("total_spent", "sum"),
        recencia_mediana = ("recency", "median"),
        freq_mediana = ("order_count", "median"),
        ticket_mediana = ("total_spent", "median"),
    )
    .reset_index()
    .sort_values(by="usuarios", ascending=False)
)

segment_profile

In [ ]:
rfm_ab = rfm_df.merge(ab_ref, on="customer_id", how="left")

rfm_ab["ab_group"].value_counts(dropna=False)

In [ ]:
ab_seg = (
    rfm_ab.groupby(["segmento", "ab_group"])
    .agg(
        usuarios = ("customer_id", "nunique"),
        freq_media = ("order_count", "mean"),
        ticket_medio = ("total_spent", "mean"),
        recencia_media = ("recency", "mean")
    )
    .reset_index()
)

ab_seg

In [ ]:
pivot = ab_seg.pivot(index="segmento", columns="ab_group")

pivot["lift_freq_%"] = (
    (pivot["freq_media"]["target"] - pivot["freq_media"]["control"]) /
    pivot["freq_media"]["control"]
) * 100

pivot["lift_ticket_%"] = (
    (pivot["ticket_medio"]["target"] - pivot["ticket_medio"]["control"]) /
    pivot["ticket_medio"]["control"]
) * 100

pivot["lift_recencia_%"] = (
    (pivot["recencia_media"]["control"] - pivot["recencia_media"]["target"]) /
    pivot["recencia_media"]["control"]
) * 100

pivot

In [ ]:
resultados_teste = []

for seg in rfm_ab["segmento"].unique():
    grupo = rfm_ab[rfm_ab["segmento"] == seg]

    # target e control
    g_target = grupo[grupo["ab_group"] == "target"]
    g_control = grupo[grupo["ab_group"] == "control"]

    # testes
    t_freq, p_freq = ttest_ind(
        g_target["order_count"],
        g_control["order_count"],
        equal_var=False,
        nan_policy='omit'
    )

    t_ticket, p_ticket = ttest_ind(
        g_target["total_spent"],
        g_control["total_spent"],
        equal_var=False,
        nan_policy='omit'
    )

    t_rec, p_rec = ttest_ind(
        g_target["recency"],
        g_control["recency"],
        equal_var=False,
        nan_policy='omit'
    )

    resultados_teste.append([
        seg,
        p_freq,
        p_ticket,
        p_rec
    ])

df_testes = pd.DataFrame(resultados_teste,
                         columns=["segmento", "p_freq", "p_ticket", "p_recencia"])

df_testes

In [ ]:
def interpret(p):
    return "✔️ significativo (p < 0.05)" if p < 0.05 else "❌ não significativo"

df_testes["freq_sig"] = df_testes["p_freq"].apply(interpret)
df_testes["ticket_sig"] = df_testes["p_ticket"].apply(interpret)
df_testes["rec_sig"] = df_testes["p_recencia"].apply(interpret)

df_testes